### 1. Total Sales by Day and Product Category

In [0]:
%sql

SELECT
  f.order_date,
  p.product_category,
  SUM(f.order_amount) AS total_sales,
  COUNT(DISTINCT f.order_id) AS orders_count
FROM `e-commerce-project`.gold.fact_orders f
JOIN `e-commerce-project`.gold.dim_products p
  ON f.product_id = p.product_id
GROUP BY f.order_date, p.product_category
ORDER BY f.order_date, total_sales DESC;


order_date,product_category,total_sales,orders_count
2025-10-08,Electronics,51900.0,1
2025-10-09,Fashion,8500.0,1
2025-10-10,Electronics,55000.0,1
2025-10-11,Home,6300.0,1
2025-10-11,Fashion,6300.0,1
2025-10-12,Home,30000.0,2
2025-10-12,Electronics,5600.0,1
2025-10-12,Fashion,5600.0,1
2025-10-13,Home,29600.0,3
2025-10-13,Electronics,20000.0,1


### 2. Customer Lifetime Value (LTV)

In [0]:
%sql
SELECT
  c.customer_id,
  c.customer_name,
  c.segment,
  SUM(f.order_amount) AS total_spent,
  COUNT(DISTINCT f.order_id) AS order_count,
  AVG(f.order_amount) AS avg_order_value
FROM `e-commerce-project`.gold.fact_orders f
JOIN `e-commerce-project`.gold.dim_customers c
  ON f.customer_id = c.customer_id
GROUP BY c.customer_id, c.customer_name, c.segment
ORDER BY total_spent DESC
LIMIT 100;


customer_id,customer_name,segment,total_spent,order_count,avg_order_value
C009,Arjun Reddy,Premium,134800.0,2,44933.333333333336
C008,Neha Nair,Retail,73000.0,3,18250.0
C003,Deepak Singh,Retail,65200.0,2,21733.333333333332
C001,Rahul Sharma,Retail,64500.0,2,16125.0
C005,Rohit Mehta,Premium,39600.0,2,13200.0
C004,Pooja Kapoor,Corporate,31750.0,3,5291.666666666667
C006,Sneha Joshi,Retail,23400.0,2,7800.0
C007,Varun Iyer,Corporate,23300.0,2,7766.666666666667
C002,Anjali Verma,Premium,15800.0,2,3950.0
C010,Divya Patil,Corporate,5500.0,3,1375.0


### 3.Payment Method Split & Success Rate

In [0]:
%sql
SELECT
  payment_method,
  COUNT(DISTINCT order_id) AS orders,
  SUM(order_amount) AS total_amount,
  ROUND(100.0 * SUM(order_amount) / SUM(SUM(order_amount)) OVER (), 2) AS pct_of_revenue
FROM `e-commerce-project`.gold.fact_orders
GROUP BY payment_method
ORDER BY total_amount DESC;


payment_method,orders,total_amount,pct_of_revenue
CARD,12,463050.0,55.13
UPI,7,218000.0,25.96
COD,11,144200.0,17.17
WALLET,1,14600.0,1.74


### 4. Courier Performance (Average Delivery Delay)

In [0]:
%sql
-- Average delay and count of delivered orders per courier
SELECT
  f.courier,
  COUNT(*) AS delivered_orders,
  AVG(f.delivery_delay_hours) AS avg_delay_hours,
  SUM(CASE WHEN f.delivery_delay_hours > 48 THEN 1 ELSE 0 END) AS breaches_over_48h
FROM `e-commerce-project`.gold.fact_orders f
LEFT JOIN `e-commerce-project`.gold.dim_courier_partners d
  ON f.courier = d.courier_name
WHERE f.shipment_status = 'DELIVERED' AND f.delivery_delay_hours IS NOT NULL
GROUP BY f.courier, d.courier_name
ORDER BY avg_delay_hours DESC;


courier,delivered_orders,avg_delay_hours,breaches_over_48h
EcomExpress,8,50.415,3
Delhivery,7,37.357142857142854,2
BlueDart,9,34.03666666666666,2
DTDC,5,15.232,0


### 5. Order Cancellation Rate

In [0]:
%sql
SELECT
  COUNT(*) AS total_orders,
  SUM(CASE WHEN shipment_status = 'CANCELED' THEN 1 ELSE 0 END) AS canceled_orders,
  ROUND(100.0 * SUM(CASE WHEN shipment_status = 'CANCELED' THEN 1 ELSE 0 END) / COUNT(*),2) AS cancel_pct
FROM `e-commerce-project`.gold.fact_orders;


total_orders,canceled_orders,cancel_pct
45,3,6.67


### 6. High-Value Order Trend Over Time

In [0]:
%sql
SELECT
  order_date,
  COUNT(DISTINCT order_id) AS high_value_orders,
  SUM(order_amount) AS total_value
FROM `e-commerce-project`.gold.fact_orders
WHERE is_high_value = true
GROUP BY order_date
ORDER BY order_date;


order_date,high_value_orders,total_value
2025-10-08,1,51900.0
2025-10-10,1,55000.0
2025-10-12,2,30000.0
2025-10-13,1,20000.0
2025-10-15,3,135000.0
2025-10-16,3,198000.0
2025-10-17,1,114800.0
2025-10-18,1,80000.0
2025-10-30,1,25000.0


### 7. Fraud Risk Distribution by Customer Segment

In [0]:
%sql
SELECT
  order_date,
  COUNT(*) AS cod_orders,
  SUM(CASE WHEN payment_status = 'PENDING' THEN 1 ELSE 0 END) AS pending_count,
  SUM(CASE WHEN shipment_status = 'CANCELED' THEN 1 ELSE 0 END) AS canceled_count,
  ROUND(100.0 * SUM(CASE WHEN shipment_status = 'CANCELED' THEN 1 ELSE 0 END) / COUNT(*), 2) AS cod_cancel_pct
FROM `e-commerce-project`.gold.fact_orders
WHERE payment_method = 'COD'
GROUP BY order_date
ORDER BY cod_cancel_pct DESC;


order_date,cod_orders,pending_count,canceled_count,cod_cancel_pct
2025-10-30,1,1,1,100.00
2025-10-14,2,0,0,0.00
2025-10-08,2,0,0,0.00
2025-10-15,3,0,0,0.00
2025-10-12,3,0,0,0.00
2025-10-19,2,0,0,0.00
2025-10-17,2,0,0,0.00


### 8) City / Region Sales Distribution

In [0]:
%sql
SELECT
  shipping_city,
  COUNT(DISTINCT order_id) AS orders,
  SUM(order_amount) AS revenue,
  ROUND(AVG(order_amount),2) AS avg_order
FROM `e-commerce-project`.gold.fact_orders
GROUP BY shipping_city
ORDER BY revenue DESC;


shipping_city,orders,revenue,avg_order
Delhi,6,236500.0,29562.5
Hyderabad,4,153350.0,21907.14
Pune,5,137200.0,19600.0
Mumbai,6,100400.0,11155.56
Chennai,3,98500.0,19700.0
Bengaluru,2,58000.0,19333.33
Kolkata,3,53400.0,13350.0
Jaipur,2,2500.0,1250.0


###10) Suspicious IPs and multi-customer IPs

In [0]:
%sql
SELECT
  ip_address,
  COUNT(DISTINCT customer_id) AS distinct_customers,
  COUNT(DISTINCT order_id) AS orders,
  SUM(CASE WHEN fraud_score > 20 THEN 1 ELSE 0 END) AS high_risk_orders
FROM `e-commerce-project`.gold.fact_orders
GROUP BY ip_address
HAVING COUNT(DISTINCT customer_id) > 1
ORDER BY distinct_customers DESC, high_risk_orders DESC;


ip_address,distinct_customers,orders,high_risk_orders
203.0.113.16,2,3,1
203.0.113.21,2,2,0
203.0.113.23,2,2,0


In [0]:
%sql
SELECT
  event_date,
  SUM(CASE WHEN fraud_score > 20 THEN 1 ELSE 0 END) OVER (ORDER BY event_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_7d_high_fraud_count
FROM (
  SELECT event_date, fraud_score
  FROM `e-commerce-project`.gold.fact_orders
) t
GROUP BY event_date, fraud_score
ORDER BY event_date;


event_date,rolling_7d_high_fraud_count
2025-10-08,1
2025-10-09,1
2025-10-10,1
2025-10-11,1
2025-10-12,1
2025-10-12,2
2025-10-12,2
2025-10-13,1
2025-10-14,1
2025-10-15,2
